In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
# API base URL
base_url = "https://search.artsmia.org/ids/"

In [3]:
# Empty dataframe
json_data = pd.DataFrame()

In [4]:
# API call function
# empty list, object variables before loop
object_id = 0
object_list = '0'
# Intial for loop - must break requests into batches of 1k, total objects in collection <132000
for i in np.arange(0,132):
    # Request size 1000
    for j in np.arange(0,1000):
        # reset and end loop to build objectlist
        if j == 999:
            object_id +=1
            # pull json data
            res = requests.get(base_url+object_list)
            # transform to json
            res = res.json()
            # Put data in dataframe
            temp_df = pd.DataFrame.from_dict(res['hits']['hits'])
            json_data = json_data.append(temp_df, ignore_index="True")
            # Reset list for next call
            object_list = str(object_id)
        else:
            # Count up object_id
            object_id +=1
            # Add new object to list
            object_list += f",{str(object_id)}"

In [41]:
json_data.head()

,_id,_index,_source,_type,_version,found
0,0,objects1,"{'id': '0', 'title': 'Air, from the series The...",object_data,38.0,True
1,1,objects1,"{'id': '1', 'title': 'Earth, from the series T...",object_data,34.0,True
2,2,objects1,"{'id': '2', 'title': 'Fire, from the series Th...",object_data,37.0,True
3,3,objects1,"{'id': '3', 'title': 'Water, from the series T...",object_data,35.0,True
4,4,objects1,"{'id': '4', 'title': 'Montana Indian Reservati...",object_data,33.0,True


In [42]:
# Pull out only objects that have ids
found_df = json_data[json_data['found']==True]

# Then split out source column into database
source_df = pd.DataFrame(col for col in found_df._source)
source_df.head()

,accessionDate,accessionHighlight,accessionHighlightText,accession_number,art_champions_text,artist,artist_suggest,catalogue_raissonne,classification,continent,...,related:videos,restricted,rights_type,role,room,see_also,signed,style,text,title
0,NaN,NaN,NaN,10.1,None,Frederick G. Smith; Artist: Formerly attribute...,"{'input': ['Frederick', 'G', 'Smith'], 'output...",None,Drawings,Europe,...,NaN,0,Public Domain,Artist,Not on View,"[0, 1, 2, 3]",Inscribed: D.G.R.,19th century,The attribution of this impressive stained gla...,"Air, from the series The Four Elements"
1,NaN,NaN,NaN,10.2,None,Frederick G. Smith; Artist: Formerly attribute...,"{'input': ['Frederick', 'G', 'Smith'], 'output...",None,Drawings,Europe,...,NaN,0,Public Domain,Artist,Not on View,"[0, 1, 2, 3]",Inscribed: [D.G.R.],19th century,The attribution of this impressive stained gla...,"Earth, from the series The Four Elements"
2,NaN,NaN,NaN,10.3,None,Frederick G. Smith; Artist: Formerly attribute...,"{'input': ['Frederick', 'G', 'Smith'], 'output...",None,Drawings,Europe,...,NaN,0,Public Domain,Artist,G352,"[0, 1, 2, 3]",Inscribed: [D.G.R.],19th century,The attribution of this impressive stained gla...,"Fire, from the series The Four Elements"
3,NaN,NaN,NaN,10.4,None,Frederick G. Smith; Artist: Formerly attribute...,"{'input': ['Frederick', 'G', 'Smith'], 'output...",None,Drawings,Europe,...,NaN,0,Public Domain,Artist,Not on View,"[0, 1, 2, 3]",Inscribed: D.G.R.,19th century,The attribution of this impressive stained gla...,"Water, from the series The Four Elements"
4,NaN,NaN,NaN,13.29,None,Walter Shirlaw,"{'input': ['Walter', 'Shirlaw'], 'output': 'Wa...",None,Drawings,North America,...,NaN,0,Public Domain,Artist,Not on View,NaN,LL,19th century,,Montana Indian Reservation I


In [43]:
# Remove deaccessioned objects
source_df_2 = source_df[source_df["deaccessioned"] != 'true']

In [44]:
# Remove unnecessary columns
cleaned_df = source_df_2[['accessionDate','accession_number', 'artist', 'classification', 'continent', 'country',
       'creditline', 'culture', 'dated', 'department', 'id', 'life_date', 'medium', 'nationality', 'object_name', 
        'provenance', 'recent', 'room', 'style', 'title']]

In [45]:
# Send dataframe to CSV for analysis
output_datafile = 'resources/Mia_objects_raw.csv'
cleaned_df.to_csv(output_datafile, encoding='utf-8')